# Классификация текстов с применением RNN

In [1]:
import pandas as pd

data = pd.read_csv('D:/PythonWork/OldZmiy/NLPWork/data/Petitions.csv')[:10000]
data.head()

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство


In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

def preprocess(text):
    text = text.lower()
    tokens = word_tokenize(text)
    filtered_tokens = [w for w in tokens if w.isalpha() and w not in stop_words]
    stemmer = SnowballStemmer("russian")
    stems = [stemmer.stem(w) for w in filtered_tokens]
    return ' '.join(stems)

data['preprocessed_text'] = data['public_petition_text'].apply(preprocess)

data.to_csv('preprocessed_dataset.csv', index=False)
data.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sheld\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\sheld\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sheld\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,public_petition_text,reason_category,preprocessed_text
0,3168490,снег на дороге,Благоустройство,снег дорог
1,3219678,очистить кабельный киоск от рекламы,Благоустройство,очист кабельн киоск реклам
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство,прос убра дерев кустарник котор вышл предел га...
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД,неудовлетворительн состоян парадн надпис двер ...
4,3336285,Граффити,Благоустройство,граффит


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(data['preprocessed_text'])

feature_names = vectorizer.get_feature_names_out()

print(tfidf_matrix.toarray())
print("Имена признаков:", feature_names)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Имена признаков: ['covid' 'dns' 'err' ... 'ясельн' 'яхтен' 'ящик']


In [4]:
categories_mapping = {}

unique_values = data['reason_category'].unique().tolist() 
mapping_dict = {val: idx+1 for idx, val in enumerate(unique_values)}  
categories_mapping['reason_category'] = mapping_dict
    
data['reason_category'] = data['reason_category'].map(mapping_dict)

data, mapping_dict

(           id                               public_petition_text  \
 0     3168490                                     снег на дороге   
 1     3219678                очистить кабельный киоск от рекламы   
 2     2963920  Просим убрать все деревья и кустарники, которы...   
 3     3374910  Неудовлетворительное состояние парадной - надп...   
 4     3336285                                           Граффити   
 ...       ...                                                ...   
 9995  3236509                                      очистите о  о   
 9996  3213091  На фасаде дома, незаконно и без разрешающих до...   
 9997  3242261  Товарный переулок. Мусор. В администрацию Цент...   
 9998  3311922  В проезжей части просел канализационный люк.\n...   
 9999  3049014                                  Надписи на будке.   
 
       reason_category                                  preprocessed_text  
 0                   1                                         снег дорог  
 1                

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, data['reason_category'], test_size=0.2, random_state=42)
y_test

6252     9
4684     2
1731     5
4742     2
4521     1
        ..
6412     1
8285     1
7853    14
1095     1
6929     2
Name: reason_category, Length: 2000, dtype: int64

In [6]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train.todense().astype(np.float32))
y_train_tensor = torch.tensor(y_train.values)

X_test_tensor = torch.tensor(X_test.todense().astype(np.float32))
y_test_tensor = torch.tensor(y_test.values)

In [7]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [8]:
batch_size = 128

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()
classifier.fit(X_train, y_train)

accuracy = classifier.score(X_test, y_test)
print(f"Точность модели: {accuracy:.2f}")

Точность модели: 0.74


## RNN

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

In [11]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        last_hidden_state = rnn_out[:, -1, :]
        return self.fc(last_hidden_state)

In [12]:
embed_dim = 7436 
hidden_size = 128
output_size = 16 
learning_rate = 0.001
num_epochs = 15

vocab_size = len(vectorizer.get_feature_names_out())

model = RNN(embed_dim, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to('cpu'), targets.to('cpu')
        
        outputs = model(inputs.unsqueeze(1))
        
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
 
        loss.backward()

        optimizer.step()
        
        total_loss += loss.item()
    
    average_loss = total_loss / len(train_dataloader)
    print(f'Эпоха {epoch+1}: Средняя потеря - {average_loss:.4f}')

Эпоха 1: Средняя потеря - 2.0089
Эпоха 2: Средняя потеря - 1.0520
Эпоха 3: Средняя потеря - 0.7348
Эпоха 4: Средняя потеря - 0.5465
Эпоха 5: Средняя потеря - 0.4237
Эпоха 6: Средняя потеря - 0.3408
Эпоха 7: Средняя потеря - 0.2796
Эпоха 8: Средняя потеря - 0.2333
Эпоха 9: Средняя потеря - 0.1985
Эпоха 10: Средняя потеря - 0.1730
Эпоха 11: Средняя потеря - 0.1504
Эпоха 12: Средняя потеря - 0.1345
Эпоха 13: Средняя потеря - 0.1210
Эпоха 14: Средняя потеря - 0.1107
Эпоха 15: Средняя потеря - 0.1029


In [13]:
from sklearn.metrics import classification_report

model.eval()

with torch.no_grad():
    outputs = model(X_test_tensor.unsqueeze(1))
    probs = outputs.softmax(dim=-1)
    predictions = probs.argmax(dim=-1)

y_true = y_test_tensor.cpu().numpy()
y_pred = predictions.cpu().numpy()

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.93      0.95      0.94      1221
           2       0.82      0.91      0.86       453
           3       0.96      0.91      0.93        55
           4       0.74      0.53      0.62        49
           5       0.80      0.89      0.84        18
           6       0.98      0.86      0.92        65
           7       0.92      0.90      0.91        39
           8       0.88      0.73      0.80        30
           9       0.78      0.30      0.44        23
          10       0.00      0.00      0.00         5
          11       1.00      0.60      0.75         5
          12       0.88      0.64      0.74        11
          13       0.88      0.47      0.61        15
          14       1.00      0.50      0.67         8
          15       1.00      0.67      0.80         3

    accuracy                           0.90      2000
   macro avg       0.84      0.66      0.72      2000
weighted avg       0.90   

d:\PythonWork\OldZmiy\NLPWork\.venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\PythonWork\OldZmiy\NLPWork\.venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\PythonWork\OldZmiy\NLPWork\.venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape

In [14]:
class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_hidden_state = lstm_out[:, -1, :]
        return self.fc(last_hidden_state)

In [15]:
embed_dim = 7436 
hidden_size = 128
output_size = 16 
learning_rate = 0.001
num_epochs = 20

vocab_size = len(vectorizer.get_feature_names_out())

model2 = MyLSTM(embed_dim, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model2.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to('cpu'), targets.to('cpu')
        
        outputs = model2(inputs.unsqueeze(1))
        
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
 
        loss.backward()

        optimizer.step()
        
        total_loss += loss.item()
    
    average_loss = total_loss / len(train_dataloader)
    print(f'Эпоха {epoch+1}: Средняя потеря - {average_loss:.4f}')

Эпоха 1: Средняя потеря - 2.4527
Эпоха 2: Средняя потеря - 1.3808
Эпоха 3: Средняя потеря - 0.9930
Эпоха 4: Средняя потеря - 0.7510
Эпоха 5: Средняя потеря - 0.5927
Эпоха 6: Средняя потеря - 0.4818
Эпоха 7: Средняя потеря - 0.3945
Эпоха 8: Средняя потеря - 0.3284
Эпоха 9: Средняя потеря - 0.2770
Эпоха 10: Средняя потеря - 0.2391
Эпоха 11: Средняя потеря - 0.2075
Эпоха 12: Средняя потеря - 0.1808
Эпоха 13: Средняя потеря - 0.1601
Эпоха 14: Средняя потеря - 0.1438
Эпоха 15: Средняя потеря - 0.1292
Эпоха 16: Средняя потеря - 0.1172
Эпоха 17: Средняя потеря - 0.1077
Эпоха 18: Средняя потеря - 0.1003
Эпоха 19: Средняя потеря - 0.0946
Эпоха 20: Средняя потеря - 0.0884


In [16]:
model2.eval()

with torch.no_grad():
    outputs = model2(X_test_tensor.unsqueeze(1))
    probs = outputs.softmax(dim=-1)
    predictions = probs.argmax(dim=-1)

y_true = y_test_tensor.cpu().numpy()
y_pred = predictions.cpu().numpy()

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.93      0.95      0.94      1221
           2       0.81      0.89      0.85       453
           3       0.94      0.89      0.92        55
           4       0.75      0.55      0.64        49
           5       0.80      0.89      0.84        18
           6       0.98      0.86      0.92        65
           7       0.92      0.92      0.92        39
           8       0.88      0.70      0.78        30
           9       0.75      0.26      0.39        23
          10       0.50      0.40      0.44         5
          11       1.00      0.60      0.75         5
          12       0.88      0.64      0.74        11
          13       0.86      0.40      0.55        15
          14       1.00      0.50      0.67         8
          15       1.00      0.67      0.80         3

    accuracy                           0.90      2000
   macro avg       0.87      0.67      0.74      2000
weighted avg       0.90   

In [17]:
class MyGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyGRU, self).__init__()
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        gru_out, _ = self.gru(x)
        last_hidden_state = gru_out[:, -1, :]
        return self.fc(last_hidden_state)

In [18]:
embed_dim = 7436 
hidden_size = 1024
output_size = 16 
learning_rate = 0.001
num_epochs = 5

vocab_size = len(vectorizer.get_feature_names_out())

model3 = MyGRU(embed_dim, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model3.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        inputs, targets = inputs.to('cpu'), targets.to('cpu')
        
        outputs = model3(inputs.unsqueeze(1))
        
        loss = criterion(outputs, targets)
 
        loss.backward()

        optimizer.step()
        
        total_loss += loss.item()
    
    average_loss = total_loss / len(train_dataloader)
    print(f'Эпоха {epoch+1}: Средняя потеря - {average_loss:.4f}')

Эпоха 1: Средняя потеря - 1.6287
Эпоха 2: Средняя потеря - 0.6458
Эпоха 3: Средняя потеря - 0.3783
Эпоха 4: Средняя потеря - 0.2484
Эпоха 5: Средняя потеря - 0.1756


In [19]:
model3.eval()

with torch.no_grad():
    outputs = model3(X_test_tensor.unsqueeze(1))
    probs = outputs.softmax(dim=-1)
    predictions = probs.argmax(dim=-1)

y_true = y_test_tensor.cpu().numpy()
y_pred = predictions.cpu().numpy()

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.93      0.95      0.94      1221
           2       0.83      0.91      0.87       453
           3       0.93      0.91      0.92        55
           4       0.80      0.57      0.67        49
           5       0.80      0.89      0.84        18
           6       0.98      0.86      0.92        65
           7       0.92      0.87      0.89        39
           8       0.88      0.77      0.82        30
           9       0.75      0.26      0.39        23
          10       0.00      0.00      0.00         5
          11       1.00      0.40      0.57         5
          12       1.00      0.64      0.78        11
          13       0.83      0.33      0.48        15
          14       1.00      0.25      0.40         8
          15       1.00      0.33      0.50         3

    accuracy                           0.90      2000
   macro avg       0.84      0.60      0.67      2000
weighted avg       0.90   

d:\PythonWork\OldZmiy\NLPWork\.venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\PythonWork\OldZmiy\NLPWork\.venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\PythonWork\OldZmiy\NLPWork\.venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape

RNN и LSTM (accuracy = 0.9) отработали лучше, чем GRU (accuracy = 0.88) 

## Модели с вниманием

### Слой внимания

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
    
    def forward(self, out, h):
        # e_i (скалярное произведение скрытых состояний с итоговым скрытым состоянием)
        es = torch.bmm(out, h.unsqueeze(2)) 
        es = es.squeeze(2)
        
        # a_i (коэффициенты внимания)
        alphas = F.softmax(es, dim=1)
        
        # a_1 * h_1 + ... + a_n * h_n (применяем коэф-ты внимания к скрытым состояниям)
        alphas = alphas.unsqueeze(1)  
        a_h_sum = torch.bmm(alphas, out)  
        a_h_sum = a_h_sum.squeeze(1)
        
        # Конкатенируем финальное состояние с полученной суммой
        c = torch.cat([h, a_h_sum], dim=1) 
        
        return c


### RNN с вниманием

In [21]:
class RNNWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNWithAttention, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.attention = Attention()
        self.fc = nn.Linear(2 * hidden_size, output_size) # (из-за конкатенации двойной размер)
    
    def forward(self, x):
        rnn_out, h = self.rnn(x)
        h = h.squeeze(0) 
        c = self.attention(rnn_out, h)  
        output = self.fc(c) 
        return output


In [22]:
embed_dim = 7436 
hidden_size = 1024
output_size = 16 
learning_rate = 0.001
num_epochs = 5

vocab_size = len(vectorizer.get_feature_names_out())

att_rnn = RNNWithAttention(embed_dim, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(att_rnn.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        inputs, targets = inputs.to('cpu'), targets.to('cpu')
        
        outputs = att_rnn(inputs.unsqueeze(1))
        
        loss = criterion(outputs, targets)
 
        loss.backward()

        optimizer.step()
        
        total_loss += loss.item()
    
    average_loss = total_loss / len(train_dataloader)
    print(f'Эпоха {epoch+1}: Средняя потеря - {average_loss:.4f}')

Эпоха 1: Средняя потеря - 1.2065
Эпоха 2: Средняя потеря - 0.4205
Эпоха 3: Средняя потеря - 0.2247
Эпоха 4: Средняя потеря - 0.1433
Эпоха 5: Средняя потеря - 0.1055


In [23]:
att_rnn.eval()

with torch.no_grad():
    outputs = att_rnn(X_test_tensor.unsqueeze(1))
    probs = outputs.softmax(dim=-1)
    predictions = probs.argmax(dim=-1)

y_true = y_test_tensor.cpu().numpy()
y_pred = predictions.cpu().numpy()

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.94      0.95      0.94      1221
           2       0.82      0.91      0.86       453
           3       0.96      0.91      0.93        55
           4       0.76      0.53      0.63        49
           5       0.80      0.89      0.84        18
           6       1.00      0.88      0.93        65
           7       0.92      0.92      0.92        39
           8       0.96      0.73      0.83        30
           9       0.80      0.35      0.48        23
          10       0.50      0.20      0.29         5
          11       1.00      0.60      0.75         5
          12       0.88      0.64      0.74        11
          13       0.88      0.47      0.61        15
          14       1.00      0.50      0.67         8
          15       1.00      0.67      0.80         3

    accuracy                           0.90      2000
   macro avg       0.88      0.68      0.75      2000
weighted avg       0.90   

По сравнению с обычной RNN, все классы предсказываются и довольно неплохо

### LSTM с вниманием 

In [ ]:
class LSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMWithAttention, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.attention = Attention()
        self.fc = nn.Linear(2 * hidden_size, output_size) # (из-за конкатенации двойной размер)
    
    def forward(self, x):
        lstm_out, (h, c) = self.lstm(x)  
        h = h.squeeze(0)  
        context = self.attention(lstm_out, h)  
        output = self.fc(context)
        return output


In [30]:
embed_dim = 7436 
hidden_size = 128
output_size = 16 
learning_rate = 0.001
num_epochs = 20

vocab_size = len(vectorizer.get_feature_names_out())

att_lstm = LSTMWithAttention(embed_dim, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(att_lstm.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to('cpu'), targets.to('cpu')
        
        outputs = att_lstm(inputs.unsqueeze(1))
        
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
 
        loss.backward()

        optimizer.step()
        
        total_loss += loss.item()
    
    average_loss = total_loss / len(train_dataloader)
    print(f'Эпоха {epoch+1}: Средняя потеря - {average_loss:.4f}')

Эпоха 1: Средняя потеря - 2.2785
Эпоха 2: Средняя потеря - 1.2176
Эпоха 3: Средняя потеря - 0.8649
Эпоха 4: Средняя потеря - 0.6174
Эпоха 5: Средняя потеря - 0.4746
Эпоха 6: Средняя потеря - 0.3710
Эпоха 7: Средняя потеря - 0.2991
Эпоха 8: Средняя потеря - 0.2465
Эпоха 9: Средняя потеря - 0.2081
Эпоха 10: Средняя потеря - 0.1778
Эпоха 11: Средняя потеря - 0.1526
Эпоха 12: Средняя потеря - 0.1326
Эпоха 13: Средняя потеря - 0.1170
Эпоха 14: Средняя потеря - 0.1046
Эпоха 15: Средняя потеря - 0.0955
Эпоха 16: Средняя потеря - 0.0888
Эпоха 17: Средняя потеря - 0.0830
Эпоха 18: Средняя потеря - 0.0779
Эпоха 19: Средняя потеря - 0.0747
Эпоха 20: Средняя потеря - 0.0709


In [31]:
att_lstm.eval()

with torch.no_grad():
    outputs = att_lstm(X_test_tensor.unsqueeze(1))
    probs = outputs.softmax(dim=-1)
    predictions = probs.argmax(dim=-1)

y_true = y_test_tensor.cpu().numpy()
y_pred = predictions.cpu().numpy()

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.94      0.94      0.94      1221
           2       0.81      0.91      0.86       453
           3       0.92      0.89      0.91        55
           4       0.77      0.55      0.64        49
           5       0.80      0.89      0.84        18
           6       0.98      0.88      0.93        65
           7       0.92      0.92      0.92        39
           8       0.88      0.70      0.78        30
           9       0.67      0.17      0.28        23
          10       0.60      0.60      0.60         5
          11       1.00      0.60      0.75         5
          12       0.78      0.64      0.70        11
          13       0.78      0.47      0.58        15
          14       1.00      0.50      0.67         8
          15       1.00      0.67      0.80         3

    accuracy                           0.90      2000
   macro avg       0.86      0.69      0.75      2000
weighted avg       0.90   

По сравнению с обычной LSTM, метрики стали выше

### GRU с вниманием

In [32]:
class GRUWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUWithAttention, self).__init__()
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.attention = Attention()
        self.fc = nn.Linear(2 * hidden_size, output_size)
    
    def forward(self, x):
        gru_out, h = self.gru(x)
        h = h.squeeze(0) 
        context = self.attention(gru_out, h) 
        output = self.fc(context)
        return output


In [33]:
embed_dim = 7436 
hidden_size = 1024
output_size = 16 
learning_rate = 0.001
num_epochs = 5

vocab_size = len(vectorizer.get_feature_names_out())

att_GRU = GRUWithAttention(embed_dim, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(att_GRU.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        inputs, targets = inputs.to('cpu'), targets.to('cpu')
        
        outputs = att_GRU(inputs.unsqueeze(1))
        
        loss = criterion(outputs, targets)
 
        loss.backward()

        optimizer.step()
        
        total_loss += loss.item()
    
    average_loss = total_loss / len(train_dataloader)
    print(f'Эпоха {epoch+1}: Средняя потеря - {average_loss:.4f}')

Эпоха 1: Средняя потеря - 1.4603
Эпоха 2: Средняя потеря - 0.5313
Эпоха 3: Средняя потеря - 0.2975
Эпоха 4: Средняя потеря - 0.1916
Эпоха 5: Средняя потеря - 0.1329


In [34]:
att_GRU.eval()

with torch.no_grad():
    outputs = att_GRU(X_test_tensor.unsqueeze(1))
    probs = outputs.softmax(dim=-1)
    predictions = probs.argmax(dim=-1)

y_true = y_test_tensor.cpu().numpy()
y_pred = predictions.cpu().numpy()

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.93      0.95      0.94      1221
           2       0.83      0.89      0.86       453
           3       0.96      0.91      0.93        55
           4       0.72      0.57      0.64        49
           5       0.80      0.89      0.84        18
           6       1.00      0.86      0.93        65
           7       0.92      0.90      0.91        39
           8       0.82      0.77      0.79        30
           9       0.82      0.39      0.53        23
          10       0.67      0.40      0.50         5
          11       1.00      0.60      0.75         5
          12       0.88      0.64      0.74        11
          13       0.88      0.47      0.61        15
          14       1.00      0.50      0.67         8
          15       1.00      0.67      0.80         3

    accuracy                           0.90      2000
   macro avg       0.88      0.69      0.76      2000
weighted avg       0.90   

По сравнению с обычной GRU, метрики стали выше